In [8]:
import pandas as pd
import numpy as np
#import optuna
from sklearn.model_selection import cross_val_score

In [12]:
data=pd.read_csv('IPL_IMB_data.csv')

In [14]:
data.head(2)

,PLAYER NAME,AGE,COUNTRY,PLAYING ROLE,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,...,HS,AVE,SR-B,SIXERS,RUNS-C,WKTS,AVE-BL,ECON,SR-BL,SOLD PRICE
0,"Abdulla, YA",2,SA,Allrounder,0,0,0,0.00,0,0.0,...,0,0.0,0.0,0,307,15,20.47,8.9,13.93,50000
1,Abdur Razzak,2,BAN,Bowler,214,18,657,71.41,185,37.6,...,0,0.0,0.0,0,29,0,0.00,14.5,0.00,50000


In [16]:
data.shape


(130, 22)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PLAYER NAME    130 non-null    object 
 1   AGE            130 non-null    int64  
 2   COUNTRY        130 non-null    object 
 3   PLAYING ROLE   130 non-null    object 
 4   T-RUNS         130 non-null    int64  
 5   T-WKTS         130 non-null    int64  
 6   ODI-RUNS-S     130 non-null    int64  
 7   ODI-SR-B       130 non-null    float64
 8   ODI-WKTS       130 non-null    int64  
 9   ODI-SR-BL      130 non-null    float64
 10  CAPTAINCY EXP  130 non-null    int64  
 11  RUNS-S         130 non-null    int64  
 12  HS             130 non-null    int64  
 13  AVE            130 non-null    float64
 14  SR-B           130 non-null    float64
 15  SIXERS         130 non-null    int64  
 16  RUNS-C         130 non-null    int64  
 17  WKTS           130 non-null    int64  
 18  AVE-BL    

In [20]:
data['AGE']=data['AGE'].astype('object')
data['CAPTAINCY EXP']=data['CAPTAINCY EXP'].astype('object')

In [22]:
data=data.drop(columns='PLAYER NAME')

In [24]:
num_data=data.select_dtypes(include=np.number)
cat_data=data.select_dtypes(exclude=np.number)

In [26]:
cat_data.head(2)

,AGE,COUNTRY,PLAYING ROLE,CAPTAINCY EXP
0,2,SA,Allrounder,0
1,2,BAN,Bowler,0


In [28]:
cat_data=pd.get_dummies(cat_data,drop_first=True,dtype=float)

In [30]:
cat_data.head(2)

,AGE_2,AGE_3,COUNTRY_BAN,COUNTRY_ENG,COUNTRY_IND,COUNTRY_NZ,COUNTRY_PAK,COUNTRY_SA,COUNTRY_SL,COUNTRY_WI,COUNTRY_ZIM,PLAYING ROLE_Batsman,PLAYING ROLE_Bowler,PLAYING ROLE_W. Keeper,CAPTAINCY EXP_1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
num_data_sc=sc.fit_transform(num_data)
num_data_sc=pd.DataFrame(num_data_sc,columns=num_data.columns)
num_data_sc.head(2)

,T-RUNS,T-WKTS,ODI-RUNS-S,ODI-SR-B,ODI-WKTS,ODI-SR-BL,RUNS-S,HS,AVE,SR-B,SIXERS,RUNS-C,WKTS,AVE-BL,ECON,SR-BL,SOLD PRICE
0,-0.657994,-0.468108,-0.703043,-2.758455,-0.686760,-1.277132,-0.839099,-1.307954,-1.693829,-3.10288,-0.745369,-0.303010,-0.099814,-0.127413,0.547597,-0.226928,-1.162826
1,-0.593006,-0.341460,-0.518927,0.009520,0.983269,0.133821,-0.839099,-1.307954,-1.693829,-3.10288,-0.745369,-0.802864,-0.790019,-1.115257,1.685233,-1.142498,-1.162826


In [34]:
data_full=pd.concat([num_data_sc,cat_data],axis=1)

In [36]:
out=data_full['SOLD PRICE']
inp=data_full.drop('SOLD PRICE',axis=1)

In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [40]:
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(inp.values,i) for i in range(inp.shape[1])]
vif['feature']=inp.columns
vif.sort_values('VIF',ascending=False).head(5)

,VIF,feature
15,45.047420,SR-BL
13,44.645799,AVE-BL
11,21.792199,RUNS-C
12,20.391780,WKTS
2,10.972636,ODI-RUNS-S


In [42]:
inp1=inp.drop(['SR-BL','AVE-BL','RUNS-C'],axis=1)
vif=pd.DataFrame()
vif['VIF']=[variance_inflation_factor(inp1.values,i) for i in range(inp1.shape[1])]
vif['feature']=inp1.columns
vif.sort_values('VIF',ascending=False).head(5)

,VIF,feature
2,10.944706,ODI-RUNS-S
6,9.481883,RUNS-S
0,8.395430,T-RUNS
7,8.363663,HS
4,6.855090,ODI-WKTS


In [18]:
import statsmodels.api as sm
inpc=sm.add_constant(inp1)
ols=sm.OLS(out,inpc)
ols_mod=ols.fit()
ols_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SOLD PRICE   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.419
Method:                 Least Squares   F-statistic:                     4.322
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.95e-08
Time:                        16:18:49   Log-Likelihood:                -133.27
No. Observations:                 130   AIC:                             324.5
Df Residuals:                     101   BIC:                             407.7
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      0.2306      0.318      0.725      0.470      -0.401       0.862
T-RUNS                    -0.2921      0.196     -1.489      0.140      -0.681       0.097
T-WKTS                    -0.0933      0.169     -0.552      0.582      -0.428       0.242
ODI-RUNS-S                 0.4246      0.222      1.910      0.059      -0.016       0.866
ODI-SR-B                   0.0017      0.087      0.019      0.985      -0.170       0.174
ODI-WKTS                   0.3344      0.176      1.900      0.060      -0.015       0.684
ODI-SR-BL                 -0.0718      0.085     -0.845      0.400      -0.240       0.097
RUNS-S                     0.2079      0.208      1.000      0.320      -0.204       0.620
HS                        -0.3427      0.194     -1.763      0.081      -0.728       0.043
AVE                        0.3528      0.171      2.063      0.042       0.013       0.692
SR-B                      -0.0781      0.100     -0.785      0.435      -0.276       0.119
SIXERS                     0.1920      0.166      1.158      0.250      -0.137       0.521
WKTS                       0.1409      0.096      1.473      0.144      -0.049       0.331
ECON                      -0.0321      0.091     -0.354      0.724      -0.212       0.148
AGE_2                     -0.5203      0.236     -2.205      0.030      -0.988      -0.052
AGE_3                     -0.7247      0.317     -2.285      0.024      -1.354      -0.096
COUNTRY_BAN               -0.8772      0.914     -0.960      0.339      -2.690       0.935
COUNTRY_ENG                1.4761      0.514      2.874      0.005       0.457       2.495
COUNTRY_IND                0.2869      0.239      1.199      0.233      -0.188       0.762
COUNTRY_NZ                 0.2035      0.361      0.564      0.574      -0.512       0.919
COUNTRY_PAK               -0.2029      0.352     -0.577      0.565      -0.901       0.495
COUNTRY_SA                -0.1403      0.272     -0.516      0.607      -0.679       0.399
COUNTRY_SL                -0.3168      0.312     -1.015      0.312      -0.936       0.302
COUNTRY_WI                -0.6285      0.390     -1.611      0.110      -1.402       0.145
COUNTRY_ZIM               -0.5560      0.881     -0.631      0.529      -2.303       1.191
PLAYING ROLE_Batsman       0.1955      0.251      0.779      0.438      -0.302       0.693
PLAYING ROLE_Bowler        0.0898      0.239      0.375      0.708      -0.385       0.564
PLAYING ROLE_W. Keeper    -0.1570      0.381     -0.412      0.681      -0.914       0.600
CAPTAINCY EXP_1            0.4239      0.247      1.718      0.089      -0.065       0.913
==============================================================================
Omnibus:                        7.050   Durbin-Watson:                   2.058
Prob(Omnibus)

In [44]:
while(len(inp1.columns)>0):
    inpc=sm.add_constant(inp1)
    ols=sm.OLS(out,inpc)
    ols_mod=ols.fit()
    
    f=ols_mod.pvalues[1:].idxmax()
    if ols_mod.pvalues[1:].max()>0.05:
        inp1=inp1.drop(f,axis=1)
    else:
        break

print('The Final Features are :', inp1.columns)    

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The Final Features are : Index(['ODI-RUNS-S', 'ODI-WKTS', 'SIXERS', 'AGE_2', 'AGE_3', 'COUNTRY_ENG',
       'COUNTRY_IND', 'COUNTRY_WI'],
      dtype='object')


In [20]:
import statsmodels.api as sm
inpc=sm.add_constant(inp1)
ols=sm.OLS(out,inpc)
ols_mod=ols.fit()
ols_mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SOLD PRICE   R-squared:                       0.462
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     13.01
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           2.09e-13
Time:                        16:18:49   Log-Likelihood:                -144.11
No. Observations:                 130   AIC:                             306.2
Df Residuals:                     121   BIC:                             332.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.2460      0.230      1.072      0.286      -0.209       0.701
ODI-RUNS-S      0.3151      0.083      3.795      0.000       0.151       0.479
ODI-WKTS        0.2521      0.071      3.543      0.001       0.111       0.393
SIXERS          0.3488      0.076      4.568      0.000       0.198       0.500
AGE_2          -0.4823      0.219     -2.206      0.029      -0.915      -0.049
AGE_3          -0.6935      0.285     -2.432      0.016      -1.258      -0.129
COUNTRY_ENG     1.7017      0.453      3.760      0.000       0.806       2.598
COUNTRY_IND     0.5269      0.152      3.466      0.001       0.226       0.828
COUNTRY_WI     -0.6852      0.330     -2.077      0.040      -1.338      -0.032
==============================================================================
Omnibus:                        7.341   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.025   Jarque-Bera (JB):                6.947
Skew:                           0.527   Prob(JB):                       0.0310
Kurtosis:                       3.416   Cond. No.                         9.03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# Regularization

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error

In [48]:
xtrain,xtest,ytrain,ytest=train_test_split(inp1,out,test_size=0.3,random_state=48)


In [50]:
lr=LinearRegression()
lr.fit(xtrain,ytrain)

ypred_train=lr.predict(xtrain)
ypred_test=lr.predict(xtest)

rmse_train=np.sqrt(mean_squared_error(ytrain,ypred_train))
rmse_test=np.sqrt(mean_squared_error(ytest,ypred_test))

r2_train=r2_score(ytrain,ypred_train)
r2_test=r2_score(ytest,ypred_test)

res_lr=[rmse_train,rmse_test,r2_train,r2_test]
res_lr

[0.6935960236384503,
 0.8439460602872784,
 0.5088381322866921,
 0.31795438052636216]

## Redge Regression

In [53]:
from sklearn.linear_model import Ridge

In [121]:
rid1 = Ridge(alpha=50.5)
rid1.fit(xtrain, ytrain)

Ridge(alpha=50.5)

In [123]:
ypred_train = rid1.predict(xtrain)
ypred_test = rid1.predict(xtest)

rmse_train = np.sqrt(mean_squared_error(ytrain, ypred_train))
rmse_test = np.sqrt(mean_squared_error(ytest, ypred_test))

r2_train = r2_score(ytrain, ypred_train)
r2_test = r2_score(ytest, ypred_test)

res_rid1 = [rmse_train,rmse_test,r2_train,r2_test]
res_rid1

[0.8300268009182421,
 0.8708347448725567,
 0.2966108189605259,
 0.2738011775418112]

## Lasso Regularization

In [126]:
from sklearn.linear_model import Lasso

In [161]:
las1 = Lasso(alpha=0.05)
las1.fit(xtrain, ytrain)

Lasso(alpha=0.05)

In [163]:
ypred_train = las1.predict(xtrain)
ypred_test = las1.predict(xtest)

rmse_train = np.sqrt(mean_squared_error(ytrain, ypred_train))
rmse_test = np.sqrt(mean_squared_error(ytest, ypred_test))

r2_train = r2_score(ytrain, ypred_train)
r2_test = r2_score(ytest, ypred_test)

res_las1 = [rmse_train,rmse_test,r2_train,r2_test]
res_las1

[0.7977374769545587,
 0.8492308266467251,
 0.35027220902360656,
 0.3093857351994026]

## Elastic Net 

In [157]:
from sklearn.linear_model import ElasticNet
ela = ElasticNet(alpha=0.1, l1_ratio = 0.5)
ela.fit(xtrain, ytrain) #If l1 ration = 0 -- ridge regularization. l1 ratio 1 -- lasso regularization 

#If l1 ratio is 0.5 which is in between; If moving towards 0 then it is ridge, other side it is lasso

ElasticNet(alpha=0.1)

In [159]:
ypred_train = ela.predict(xtrain)
ypred_test = ela.predict(xtest)

rmse_train = np.sqrt(mean_squared_error(ytrain, ypred_train))
rmse_test = np.sqrt(mean_squared_error(ytest, ypred_test))

r2_train = r2_score(ytrain, ypred_train)
r2_test = r2_score(ytest, ypred_test)

res_ela = [rmse_train,rmse_test,r2_train,r2_test]
res_ela

[0.8109053171740085,
 0.8540860296462246,
 0.3286457399355487,
 0.3014664336411256]

In [167]:
result = pd.DataFrame()
result['LR'] = res_lr
result['Ridge_alpha_1'] = res_rid1
result['Ridge_alpha_4'] = res_rid4
result['Ridge_alpha_10'] = res_rid10
result['Ridge_alpha_50'] = res_rid50
result['Lasso_alpha_0.05'] = res_les
result['ElasticNet_alpha_0.1_11=0.5'] = res_enet
result.index = ['rmse_train','rmse_test','r2_train','r2_test']


NameError: name 'res_rid4' is not defined

In [171]:
coef = pd.DataFrame(lr.coef_, index = xtrain.columns, columns=['LR'])
coef

,LR
ODI-RUNS-S,0.278054
ODI-WKTS,0.290678
SIXERS,0.427187
AGE_2,-0.284312
AGE_3,-0.434347
COUNTRY_ENG,1.760034
COUNTRY_IND,0.619800
COUNTRY_WI,-0.919388


In [173]:
coef['Ridge_alpha_1'] = rid1.coef_
coef['Ridge_alpha_4'] = rid4.coef_
coef['Ridge_alpha_50'] = rid50.coef_
coef['Lasso_alpha_0.05'] = las1.coef_
coef['ElasticNet_alpha_0.05'] = ela.coef_


NameError: name 'rid4' is not defined

In [175]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rid = Ridge()
param = ('alpha' : [0.000001, 0.00001, 0.000, 0.001, 0.01, 0.1, 1, 2, 5, 8, 10, 50, 60, 70, 90, 100])

In [183]:
from sklearn.linear_model import SGDRegressor

In [185]:
sgd = SGDRegressor(random_state = 10)
linreg_withSGD

In [187]:
coef['SGD'] = linreg_with_SGD.coef_

NameError: name 'linreg_with_SGD' is not defined